In [1]:
import requests
from bs4 import BeautifulSoup
import openpyxl
import pandas as pd
import re  # 숫자추출을 위한 라이브러리
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pyperclip # 자동 로그인을 위한 변수 복사 라이브러리
from time import sleep


In [2]:
def create_excel(time, rank, server) : 
# 엑셀 만들기
    wb = openpyxl.Workbook()

    # 워크시트 만들기
    ws = wb.active
    ws.title = f"최근 {time}일 {server}서버의 {rank}랭크"

    # 데이터 추가하기
    ws.append(['직업', '덱유형', '승률', '점유율', '게임수'])

    # 엑셀 저장하기
    wb.save('./hs_meta.xlsx')



In [3]:
def create_sheet(wb):
  # 기존 'Matchups' 시트가 존재하면 삭제
  if 'Matchups' in wb.sheetnames:
    wb.remove(wb['Matchups'])
  # 새로운 시트 생성하고 반환
  ws = wb.create_sheet(title='Matchups')
  wb.save('./hs_meta.xlsx')
  return ws

In [4]:
def driver_in(): 
# ChromeOptions 설정
  chrome_options = Options()
  chrome_options.add_experimental_option("detach", True)
  chrome_options.add_argument("--incognito")
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')

  # ChromeDriverManager를 사용하여 Chrome 드라이버를 초기화합니다.
  driver = webdriver.Chrome(service=ChromeService(executable_path=ChromeDriverManager().install()), options=chrome_options)

  # url 열기
  url = "https://hsreplay.net/account/login"
  driver.get(url)
  
  # 엘리먼트가 나타날 때까지 대기합니다 (최대 10초 대기)
  try:
      element_present = EC.presence_of_element_located((By.CLASS_NAME, "promo-button"))
      WebDriverWait(driver, 10).until(element_present)
  except Exception as e:
      print(f"엘리먼트 대기 중 예외 발생1: {e}")
      driver.quit()

  btn = driver.find_element(By.CLASS_NAME,"promo-button")
  btn.click()

  # 엘리먼트가 나타날 때까지 대기합니다 (최대 10초 대기)
  try:
      element_present = EC.presence_of_element_located((By.CLASS_NAME, "input-block"))
      WebDriverWait(driver, 10).until(element_present)
  except Exception as e:
      print(f"엘리먼트 대기 중 예외 발생2: {e}")
      driver.quit()

  id = driver.find_elements(By.CLASS_NAME, "input-block")[0]
  pyperclip.copy("tlthr763@naver.com") # ID 복사
  id.send_keys(Keys.CONTROL, 'v') # ID 붙여넣기
  pw = driver.find_elements(By.CLASS_NAME, "input-block")[1]
  pyperclip.copy("qwer123657.") # 패스워드 복사
  pw.send_keys(Keys.CONTROL, 'v') # 패스워드 붙여넣기
  log_btn = driver.find_element(By.CLASS_NAME, "submit-button")
  log_btn.click()

  return driver, chrome_options

In [5]:
# 입력값 받기
time = int(input("검색할 기간을 입력하세요.\n[최근7일 : 7 // 최근 3일 : 3 // 최근 1일 : 1 // 최신 패치 : 0]"))
rank = input("검색할 랭크를 입력하세요.\n[전구간 : A // 브론즈 : B // 실버 : S // 골드 : G // 플래티넘 : P // 다이아 : D // 전설 : L // 상위1000 : 1000]")
server = input("검색할 서버를 입력하세요.\n[전서버 : A // 아시아 : AP // 유럽 : EU // 북미 : US]")

# 검색 범위 딕셔너리 정의
time_dict={7 : "LAST_7_DAY", 3 :  "LAST_3_DAY", 1 : "LAST_1_DAY", 0 : ""}
rank_dict={"A" : "", "B" :  "BRONZE", "S" : "SILVER", "G" :  "GOLD", "P" :  "PLATINUM", "D" :  "DIAMOND", "L" :  "LEGEND", "1000" :  "TOP_1000_LEGEND"}
server_dict={"A" : "", "AP" :  "REGION_KR", "EU" : "REGION_EU", "US" : "REGION_US"}

# 엑셀 생성 함수
create_excel(time, rank_dict[rank], server_dict[server])


# 웹브라우저 진입 함수
driver, chrome_opteions = driver_in()
sleep(1) # 로그인 진입을 위한 대기시간 1초


# URL 열기
if rank_dict[rank]=="":
  url = f"https://hsreplay.net/meta/#tab=archetypes&timeFrame={time_dict[time]}&region={server_dict[server]}"
else:
  url = f"https://hsreplay.net/meta/#tab=archetypes&timeFrame={time_dict[time]}&rankRange={rank_dict[rank]}&region={server_dict[server]}"
driver.get(url)

# 엘리먼트가 나타날 때까지 대기합니다 (최대 10초 대기)
try:
  element_present = EC.presence_of_element_located((By.CLASS_NAME, 'class-box-container'))
  WebDriverWait(driver, 10).until(element_present)
except Exception as e:
  print(f"엘리먼트 대기 중 예외 발생3: {e}")
  driver.quit()



# BeautifulSoup을 사용하여 HTML 파싱
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 직업 클래스
elements = driver.find_elements(By.CSS_SELECTOR, '.class-box')

# 엑셀 열기
wb = openpyxl.load_workbook('./hs_meta.xlsx')
ws = wb.active

# 데이터를 저장할 리스트 생성
data = []

# 직업별 정보 저장 루프
for element in elements:
    # 직업 가져오기
    hero = element.find_element(By.CSS_SELECTOR, '.box-title').text
    #print(f"직업 {hero}")
    
    # 해당 직업에 대한 덱 유형과 정보 클래스
    deck_elements = element.find_elements(By.CSS_SELECTOR, '.table-row-header')
    stats_elements = element.find_elements(By.CSS_SELECTOR, '.table-cell')


    for deck_element in deck_elements:
      # 덱 유형 가져오기
      type_name = deck_element.find_element(By.CSS_SELECTOR, '.tooltip-wrapper').text
      #print(f"덱유형 {type_name}")

      # 승률, 점유율, 게임 수 가져오기
      win_rate = stats_elements[0].text
      popularity = stats_elements[1].text
      games = stats_elements[2].text
      #print(f"승률: {win_rate}, 점유율: {popularity}, 게임 수: {games}")
    
      # 데이터를 리스트에 추가
      data.append([hero, type_name, win_rate, popularity, games])

      # stats_elements에서 현재 처리한 스탯 정보를 제거
      stats_elements = stats_elements[3:]

# .xlsx 형식으로 엑셀에 추가
for row in data:
    ws.append(row)

# .xlsx 형식으로 엑셀 저장
wb.save('./hs_meta.xlsx')


In [6]:
# 창의 너비와 높이 설정 
driver.set_window_size(1592, 1039)

# '매치업' 탭 클릭
matchup_tab = driver.find_element(By.ID, 'tab-matchups')
matchup_tab.click()

# 엑셀 새로운 시트 생성
ws_match = create_sheet(wb)

# 덱 이름을 저장할 리스트 생성
data2 = ["",]

# 덱 이름 출력
arche_elements = driver.find_elements(By.CLASS_NAME, 'matchup-row-header')
for arche in arche_elements:
    try :
      isarche=arche.find_element(By.CLASS_NAME, 'archetype-name')
      deck_type = isarche.text 
      data2.append(deck_type)
    except:
      notarche=arche.find_element(By.TAG_NAME, 'aside')
      deck_type = notarche.text
      data2.append(deck_type)
            
# .xlsx 형식으로 엑셀에 추가
ws_match.append(data2)
data2.pop(0)

# 개별 항목을 다시 추가
for row in data2:
    ws_match.append([row])

# 표 데이터를 저장할 리스트 생성
data3 = []

# 표 클래스 찾기
rate_matrix = driver.find_elements(By.CLASS_NAME, 'matchup-cell')
count = 0
row_index = 2  # B2 셀부터 시작

for rate in rate_matrix:
    rate_text = rate.find_element(By.CLASS_NAME, 'tooltip-wrapper')
    rate_element = rate_text.text

    # rate_element를 적절히 처리하여 데이터 추출
    # 예를 들어, rate_element를 콤마(,)로 분리하여 리스트로 변환
    cell_data = rate_element.split(',')

    # data3 리스트에 데이터 추가
    data3.extend(cell_data)

    # 14개의 데이터가 모일 때마다 엑셀에 행 추가
    if len(data3) == 14:
        # 행, 열 수에 맞게 엑셀에 추가
        for col_index, value in enumerate(data3):
            ws_match.cell(row=row_index, column=col_index + 2, value=value)
        
        data3.clear()  # 데이터를 비워줌
        row_index += 1
        count += 1

    if count == 14:
        break

# .xlsx 형식으로 엑셀 저장
wb.save('./hs_meta.xlsx')
wb.close()


In [10]:
def save_csv():
  # 기존 엑셀 파일 열기
  wb = openpyxl.load_workbook('./hs_meta.xlsx')

  # '매치업' 시트 선택
  ws_matchups = wb['Matchups']

  # 시트의 내용을 데이터프레임으로 변환
  df_matchups = pd.DataFrame(ws_matchups.values)
  print(df_matchups)

  # CSV 파일로 저장
  df_matchups.to_csv('./hs_meta_matchups.csv', encoding='utf-8-sig', header=0)

  # 기존 엑셀 파일 닫기
  wb.close()

In [8]:
# 데이터를 포함한 리스트에서 DataFrame을 생성
df = pd.DataFrame(data, columns=['Hero', 'Deck', 'Winrate', 'Popular', 'Games'])

# DataFrame을 UTF-8-sig 인코딩으로 CSV 파일로 저장
df.to_csv('./hs_meta.csv', encoding='utf-8-sig', index=False)

# Matchups 시트를 UTF-8-sig 인코딩으로 CSV 파일로 저장
save_csv()

                     0                       1   \
0                  None                  용 드루이드   
1                용 드루이드  용 드루이드\nMirror matchup   
2   Plague Death Knight                  40.75%   
3               어그로 성기사                  69.03%   
4                컨트롤 전사                  41.72%   
5        Mining Warlock                  43.09%   
6             하이랜더 드루이드                  50.20%   
7             부정 죽음의 기사                  23.67%   
8               사냥개 사냥꾼                  48.44%   
9      Showdown Paladin                  69.77%   
10         Rainbow Mage                  39.75%   
11            어그로 악마사냥꾼                  38.39%   
12                 None                  36.38%   
13               컨트롤 사제                  45.05%   
14            혈기 죽음의 기사                  27.58%   
15                 None                    None   
16                 None                    None   
17                 None                    None   
18                 None        

In [11]:
save_csv()

                     0                       1   \
0                  None                  용 드루이드   
1                용 드루이드  용 드루이드\nMirror matchup   
2   Plague Death Knight                  40.75%   
3               어그로 성기사                  69.03%   
4                컨트롤 전사                  41.72%   
5        Mining Warlock                  43.09%   
6             하이랜더 드루이드                  50.20%   
7             부정 죽음의 기사                  23.67%   
8               사냥개 사냥꾼                  48.44%   
9      Showdown Paladin                  69.77%   
10         Rainbow Mage                  39.75%   
11            어그로 악마사냥꾼                  38.39%   
12               기타 성기사                  36.38%   
13               컨트롤 사제                  45.05%   
14            혈기 죽음의 기사                  27.58%   

                                     2                        3   \
0                   Plague Death Knight                  어그로 성기사   
1                                59.24%        